In [4]:
# Importar as bibliotecas
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_moons, make_circles, make_blobs
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, accuracy_score, roc_auc_score, roc_curve, RocCurveDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
from os.path import join
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, accuracy_score, roc_auc_score, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [7]:
# Importar os datasets
bkg = pd.read_hdf(join('C:\\Users\\Kiki - User\\Documents\\Estágio LIP 2023\\DataSet', 'bkg_pythia_sanitised_features.h5'))
fcnc = pd.read_hdf(join('C:\\Users\\Kiki - User\\Documents\\Estágio LIP 2023\\DataSet', 'fcnc_pythia_sanitised_features.h5'))

data = pd.concat([bkg, fcnc])

data.shape

(1093038, 83)

In [9]:
data

,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,FatJet1_Phi,FatJet1_Tau1,FatJet1_Tau2,FatJet1_Tau3,FatJet1_Tau4,FatJet1_Tau5,...,Jet8_BTag,Jet8_Eta,Jet8_Mass,Jet8_PT,Jet8_Phi,Jet9_BTag,Jet9_Eta,Jet9_Mass,Jet9_PT,Jet9_Phi
0,2,-1.988600,52.710262,229.350952,0.728242,36.148926,23.039709,16.949991,14.424411,12.000529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0.528382,61.115589,315.538910,-0.863614,32.592808,22.366640,16.285843,13.938633,11.180016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1.479911,98.012802,251.109573,-3.133624,90.252274,33.646885,30.612156,27.973904,23.729696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,0.926899,-0.000007,240.909348,0.835656,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0.781194,72.234299,206.020386,-0.320449,48.886372,20.743645,16.572512,13.070706,11.269534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90543,0,-1.104331,98.255119,616.717834,2.647943,73.691048,2.691046,0.611649,0.298947,0.067618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90544,1,2.446506,93.868454,345.995178,1.510921,68.885880,1.313736,0.359513,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90545,1,0.298318,103.052811,715.299622,-0.064837,62.022102,3.699583,1.385654,0.390390,0.148895,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90546,1,0.034329,100.949005,452.415161,2.614630,48.379311,5.810863,2.914674,1.750562,0.716167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Nomear os gen_label (que nos indica o sinal e o fundo) de label e gen_x (que nos dá os pesos) de weights
data.rename(columns={'gen_label': 'label', 'gen_xsec': 'weights'}, inplace=True)

# Agora definimos o fundo de 0 e o sinal de 1
data['label'] = data['label'].map({'bkg': 0, 'signal':1})

# Remove NaN
data = data.dropna(axis=1)

# Remove all columns with 'gen' (agora aqui já não temos as labels nem os pesos)
data = data[[x for x in data.columns if 'gen' not in x]]

In [11]:
# Transformar o sinal de fundo e de sinal em percentagem 
len_bkg, len_fcnc = len(bkg), len(fcnc)
bkg_percentage = len_bkg/(len_bkg + len_fcnc)
fcnc_percentage = len_fcnc/(len_bkg + len_fcnc)
bkg_percentage, fcnc_percentage# Limitar os dados que temos 
n_total_samples = 100000
n_bkg_samples = int(n_total_samples * bkg_percentage)
n_fcnc_samples = int(n_total_samples * fcnc_percentage)

n_bkg_samples, n_fcnc_samples

(0.9171593302337155, 0.08284066976628443)

In [12]:
# Limitar os dados que temos 
n_total_samples = 100000
n_bkg_samples = int(n_total_samples * bkg_percentage)
n_fcnc_samples = int(n_total_samples * fcnc_percentage)

n_bkg_samples, n_fcnc_samples

(91715, 8284)

In [13]:
bkg = data[data["label"] == 0].sample(n_bkg_samples, random_state=42)
fcnc = data[data["label"] == 1].sample(n_fcnc_samples, random_state=42)

# Dividir entre treino, validação e teste 
sig_train, sig_validate, sig_test = np.split(fcnc, [int((1 / 3) * len(fcnc)), int((2 / 3) * len(fcnc))])
bkg_train, bkg_validate, bkg_test = np.split(bkg, [int((1 / 3) * len(bkg)), int((2 / 3) * len(bkg))])

# Criar os datasets de treino, validação e teste (concatenar os anteriores)
train_data = pd.concat([sig_train, bkg_train])
val_data = pd.concat([sig_validate, bkg_validate])
test_data = pd.concat([sig_test, bkg_test])

In [14]:
# Separar as variáveis em vários conjuntos de dados
feature_train, label_train, weight_train = train_data.drop(columns=['label', 'weights']), train_data['label'], train_data['weights']
feature_val, label_val, weight_val = val_data.drop(columns=['label', 'weights']), val_data['label'], val_data['weights']
feature_test, label_test, weight_test = test_data.drop(columns=['label', 'weights']), test_data['label'], test_data['weights']

In [15]:
# Percentagem de sinal e fundo
label_train.value_counts(normalize=True)

weight_train[label_train == 0].sum(), weight_train[label_train == 1].sum()

(48945.286000138076, 3.547331848082502)

In [20]:
# Normalizar os pesos (fazemos isto porque existem muito mais dados de background do que sinal)
weight_train[label_train == 1] = (weight_train[label_train == 1] /weight_train[label_train == 1].sum()) * weight_train.shape[0] / 2
weight_train[label_train == 0] = (weight_train[label_train == 0] / weight_train[label_train == 0].sum()) * weight_train.shape[0] / 2

weight_val[label_val == 1] = (weight_val[label_val == 1] / weight_val[label_val == 1].sum()) * weight_val.shape[0] / 2
weight_val[label_val == 0] = (weight_val[label_val == 0] / weight_val[label_val == 0].sum()) * weight_val.shape[0] / 2

weight_test[label_test == 1] = (weight_test[label_test == 1] / weight_test[label_test == 1].sum()) * weight_test.shape[0] / 2
weight_test[label_test == 0] = (weight_test[label_test == 0] / weight_test[label_test == 0].sum()) * weight_test.shape[0] / 2

In [26]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [37]:
# Create an instance of SVM
svm = SVC(kernel="rbf", probability=True)

# Train the SVM model
svm.fit(feature_train, label_train, sample_weight=weight_train)

# Separate the features, labels, and weights for the validation data
feature_val = val_data.drop(columns=['label', 'weights'])
label_val = val_data['label']
weight_val = val_data['weights']

# Predict labels for the validation data
val_predictions = svm.predict(feature_val)
val_test_scores = svm.predict_proba(feature_test)
val_test_scores = val_test_scores[:, 1]

# Calculate accuracy and ROC AUC score for validation data
accuracy = accuracy_score(label_val, val_predictions)
roc_auc = roc_auc_score(label_val, val_predictions, sample_weight=weight_val)

# Print the accuracy and ROC AUC score
print("Validation Accuracy:", accuracy)
print("Validation ROC AUC Score:", roc_auc)

Validation Accuracy: 0.7434374343743437
Validation ROC AUC Score: 0.8272738164522248


In [39]:
fpr, tpr, thresholds = roc_curve(label_val, val_test_scores,sample_weight=weight_val.ravel())

# Plot the ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], 'k--', color='orange')  
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [33333, 33334, 33333]